In [1]:

#DATA LOADING
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

DATA_PATH = Path('/kaggle/input/mallorn-dataset')

train_log = pd.read_csv(DATA_PATH / 'train_log.csv')
test_log = pd.read_csv(DATA_PATH / 'test_log.csv')

print(f"Train: {train_log.shape}, Test: {test_log.shape}")
print(f"TDE ratio: {train_log['target'].mean():.4f}")

Train: (3043, 8), Test: (7135, 7)
TDE ratio: 0.0486


In [2]:
#ENHANCED FEATURE EXTRACTOR + TDE PHYSICS
from scipy.stats import skew, kurtosis
from scipy.signal import find_peaks
import gc

class EnhancedFeatureExtractor:
    """Physics-informed + Statistical features"""
    
    def __init__(self):
        self.filters = ['u', 'g', 'r', 'i', 'z', 'y']
    
    def extract_features(self, lc_data, obj_id, metadata=None):
        features = {'object_id': obj_id}
        
        if metadata is not None:
            features['Z'] = metadata.get('Z', np.nan)
            features['EBV'] = metadata.get('EBV', np.nan)
        
        obj_lc = lc_data[lc_data['object_id'] == obj_id].copy()
        
        if len(obj_lc) == 0:
            return features
        
        obj_lc = obj_lc.sort_values('Time (MJD)').reset_index(drop=True)
        
        # === CORE FEATURES ===
        features.update(self._global_features(obj_lc))
        features.update(self._per_filter_features(obj_lc))
        features.update(self._multiband_features(obj_lc))
        features.update(self._tde_specific_features(obj_lc))
        features.update(self._statistical_features(obj_lc))
        features.update(self._tde_powerlaw_features(obj_lc))  # NEW!
        
        return features
    
    def _global_features(self, lc):
        f = {}
        f['n_obs'] = len(lc)
        f['n_filters'] = lc['Filter'].nunique()
        f['duration'] = lc['Time (MJD)'].max() - lc['Time (MJD)'].min()
        
        time_diffs = lc['Time (MJD)'].diff().dropna()
        f['cadence_mean'] = time_diffs.mean()
        f['cadence_std'] = time_diffs.std()
        f['cadence_median'] = time_diffs.median()
        
        # Flux stats
        f['flux_mean'] = lc['Flux'].mean()
        f['flux_std'] = lc['Flux'].std()
        f['flux_median'] = lc['Flux'].median()
        f['flux_max'] = lc['Flux'].max()
        f['flux_min'] = lc['Flux'].min()
        f['flux_range'] = f['flux_max'] - f['flux_min']
        f['flux_iqr'] = lc['Flux'].quantile(0.75) - lc['Flux'].quantile(0.25)
        f['flux_mad'] = np.median(np.abs(lc['Flux'] - lc['Flux'].median()))
        
        # SNR
        snr = np.abs(lc['Flux'] / (lc['Flux_err'] + 1e-10))
        f['snr_mean'] = snr.mean()
        f['snr_median'] = snr.median()
        f['snr_max'] = snr.max()
        
        # Detections
        detections = lc[lc['Flux'] > 3 * lc['Flux_err']]
        f['n_detections'] = len(detections)
        f['detection_rate'] = len(detections) / len(lc)
        
        return f
    
    def _per_filter_features(self, lc):
        f = {}
        
        for filt in self.filters:
            filt_lc = lc[lc['Filter'] == filt].copy()
            prefix = f'{filt}_'
            
            if len(filt_lc) == 0:
                f[prefix + 'n_obs'] = 0
                continue
            
            filt_lc = filt_lc.sort_values('Time (MJD)').reset_index(drop=True)
            
            f[prefix + 'n_obs'] = len(filt_lc)
            f[prefix + 'flux_mean'] = filt_lc['Flux'].mean()
            f[prefix + 'flux_std'] = filt_lc['Flux'].std()
            f[prefix + 'flux_max'] = filt_lc['Flux'].max()
            f[prefix + 'flux_min'] = filt_lc['Flux'].min()
            f[prefix + 'flux_range'] = f[prefix + 'flux_max'] - f[prefix + 'flux_min']
            
            # Variability
            if len(filt_lc) > 3:
                f[prefix + 'stetson_j'] = self._stetson_j(filt_lc['Flux'].values, filt_lc['Flux_err'].values)
                f[prefix + 'excess_var'] = self._excess_variance(filt_lc['Flux'].values, filt_lc['Flux_err'].values)
            
            # Peak analysis
            if len(filt_lc) > 2:
                peak_idx = filt_lc['Flux'].idxmax()
                if pd.notna(peak_idx):
                    f[prefix + 'time_to_peak'] = filt_lc.loc[peak_idx, 'Time (MJD)'] - filt_lc['Time (MJD)'].min()
                    f[prefix + 'time_from_peak'] = filt_lc['Time (MJD)'].max() - filt_lc.loc[peak_idx, 'Time (MJD)']
                    
                    if f[prefix + 'time_from_peak'] > 0:
                        f[prefix + 'rise_fall_ratio'] = f[prefix + 'time_to_peak'] / f[prefix + 'time_from_peak']
        
        return f
    
    def _multiband_features(self, lc):
        f = {}
        
        # Colors
        for f1, f2 in [('u', 'g'), ('g', 'r'), ('r', 'i'), ('i', 'z'), ('u', 'r')]:
            lc1 = lc[lc['Filter'] == f1]
            lc2 = lc[lc['Filter'] == f2]
            
            if len(lc1) > 0 and len(lc2) > 0:
                f[f'color_{f1}_{f2}_mean'] = lc1['Flux'].mean() - lc2['Flux'].mean()
                f[f'color_{f1}_{f2}_max'] = lc1['Flux'].max() - lc2['Flux'].max()
        
        # Peak synchronization
        peak_times = []
        for filt in self.filters:
            filt_lc = lc[lc['Filter'] == filt]
            if len(filt_lc) > 0:
                peak_idx = filt_lc['Flux'].idxmax()
                if pd.notna(peak_idx):
                    peak_times.append(filt_lc.loc[peak_idx, 'Time (MJD)'])
        
        if len(peak_times) > 1:
            f['peak_time_std'] = np.std(peak_times)
            f['peak_time_range'] = max(peak_times) - min(peak_times)
        
        return f
    
    def _tde_specific_features(self, lc):
        """TDE-specific indicators"""
        f = {}
        
        # r-band analysis (best sampled)
        r_lc = lc[lc['Filter'] == 'r'].copy()
        
        if len(r_lc) > 10:
            r_lc = r_lc.sort_values('Time (MJD)').reset_index(drop=True)
            
            peak_idx = r_lc['Flux'].idxmax()
            if pd.notna(peak_idx):
                # Parabolic fits
                rise = r_lc[r_lc.index <= peak_idx]
                decline = r_lc[r_lc.index >= peak_idx]
                
                if len(rise) > 3:
                    f['r_rise_parabolic_fit'] = self._fit_polynomial(rise['Time (MJD)'].values, rise['Flux'].values, 2)
                
                if len(decline) > 3:
                    f['r_decline_parabolic_fit'] = self._fit_polynomial(decline['Time (MJD)'].values, decline['Flux'].values, 2)
                
                # Symmetry
                f['r_peak_position'] = peak_idx / len(r_lc)
        
        # Peak counts (TDEs: 1 peak, AGN: many)
        for filt in ['r', 'g', 'i']:
            filt_lc = lc[lc['Filter'] == filt]
            if len(filt_lc) > 5:
                peaks, _ = find_peaks(filt_lc['Flux'].values, prominence=filt_lc['Flux'].std())
                f[f'{filt}_n_peaks'] = len(peaks)
        
        return f
    
    def _tde_powerlaw_features(self, lc):
        """TDE Power-law signatures: t^5/3 rise, t^-5/3 decline"""
        f = {}
        
        for filt in ['r', 'g', 'i']:
            filt_lc = lc[lc['Filter'] == filt].copy()
            
            if len(filt_lc) < 15:
                continue
            
            filt_lc = filt_lc.sort_values('Time (MJD)').reset_index(drop=True)
            peak_idx = filt_lc['Flux'].idxmax()
            
            if pd.notna(peak_idx):
                # RISE POWER-LAW
                rise = filt_lc[filt_lc.index <= peak_idx]
                if len(rise) > 5:
                    t_norm = rise['Time (MJD)'] - rise['Time (MJD)'].min() + 1
                    valid = (rise['Flux'] > 0) & (t_norm > 0)
                    
                    if valid.sum() > 4:
                        log_t = np.log10(t_norm[valid].values)
                        log_f = np.log10(rise['Flux'][valid].values)
                        
                        try:
                            coeffs = np.polyfit(log_t, log_f, 1)
                            f[f'{filt}_rise_powerlaw'] = coeffs[0]  # Should be ~1.67 for TDE
                            
                            pred = np.polyval(coeffs, log_t)
                            r2 = 1 - np.sum((log_f - pred)**2) / (np.sum((log_f - log_f.mean())**2) + 1e-10)
                            f[f'{filt}_rise_powerlaw_r2'] = r2
                        except:
                            pass
                
                # DECLINE POWER-LAW
                decline = filt_lc[filt_lc.index >= peak_idx]
                if len(decline) > 5:
                    t_norm = decline['Time (MJD)'] - decline['Time (MJD)'].min() + 1
                    valid = (decline['Flux'] > 0) & (t_norm > 0)
                    
                    if valid.sum() > 4:
                        log_t = np.log10(t_norm[valid].values)
                        log_f = np.log10(decline['Flux'][valid].values)
                        
                        try:
                            coeffs = np.polyfit(log_t, log_f, 1)
                            f[f'{filt}_decline_powerlaw'] = coeffs[0]  # Should be ~-1.67 for TDE
                            
                            pred = np.polyval(coeffs, log_t)
                            r2 = 1 - np.sum((log_f - pred)**2) / (np.sum((log_f - log_f.mean())**2) + 1e-10)
                            f[f'{filt}_decline_powerlaw_r2'] = r2
                        except:
                            pass
        
        return f
    
    def _statistical_features(self, lc):
        """Advanced statistical features"""
        f = {}
        
        flux = lc['Flux'].values
        
        # Higher moments
        f['flux_skewness'] = skew(flux) if len(flux) > 2 else np.nan
        f['flux_kurtosis'] = kurtosis(flux) if len(flux) > 2 else np.nan
        
        # Percentiles
        f['flux_p05'] = np.percentile(flux, 5)
        f['flux_p95'] = np.percentile(flux, 95)
        f['flux_p10_p90_ratio'] = np.percentile(flux, 90) / (np.percentile(flux, 10) + 1e-10)
        
        # Amplitude features
        f['amplitude'] = (f['flux_p95'] - f['flux_p05']) / 2
        f['beyond_1std'] = np.sum(np.abs(flux - flux.mean()) > flux.std()) / len(flux)
        
        return f
    
    # Helper methods
    def _stetson_j(self, flux, flux_err):
        try:
            if len(flux) < 2:
                return np.nan
            mean_flux = np.mean(flux)
            delta = (flux - mean_flux) / (flux_err + 1e-10)
            delta = delta[np.isfinite(delta)]
            if len(delta) < 2:
                return np.nan
            result = np.mean(np.sign(delta[:-1] * delta[1:]) * np.sqrt(np.abs(delta[:-1] * delta[1:])))
            return result if np.isfinite(result) else np.nan
        except:
            return np.nan
    
    def _excess_variance(self, flux, flux_err):
        try:
            mean_flux = np.mean(flux)
            if mean_flux == 0:
                return np.nan
            variance = np.var(flux)
            mean_err_sq = np.mean(flux_err**2)
            excess = max(0, (variance - mean_err_sq) / mean_flux**2)
            return excess if np.isfinite(excess) else np.nan
        except:
            return np.nan
    
    def _fit_polynomial(self, x, y, degree):
        try:
            mask = np.isfinite(x) & np.isfinite(y)
            x, y = x[mask], y[mask]
            if len(x) < degree + 1:
                return np.nan
            coeffs = np.polyfit(x, y, degree)
            y_pred = np.polyval(coeffs, x)
            ss_res = np.sum((y - y_pred)**2)
            ss_tot = np.sum((y - np.mean(y))**2)
            r2 = 1 - (ss_res / ss_tot) if ss_tot > 0 else 0
            return r2 if np.isfinite(r2) else np.nan
        except:
            return np.nan

def load_split_lightcurves(split_name, data_type='train'):
    file_path = DATA_PATH / split_name / f'{data_type}_full_lightcurves.csv'
    return pd.read_csv(file_path)

✓ Enhanced Feature Extractor with TDE Power-law features ready!


In [3]:

#FEATURE EXTRACTION
from tqdm.auto import tqdm
import time

extractor = EnhancedFeatureExtractor()
all_splits = [f'split_{i:02d}' for i in range(1, 21)]

def extract_split_features(split_name):
    lc_train = load_split_lightcurves(split_name, 'train')
    lc_test = load_split_lightcurves(split_name, 'test')
    
    train_meta = train_log[train_log['split'] == split_name]
    test_meta = test_log[test_log['split'] == split_name]
    
    train_feats = []
    for obj_id in tqdm(train_meta['object_id'], desc=f'{split_name} train', leave=False):
        meta = train_meta[train_meta['object_id'] == obj_id].iloc[0].to_dict()
        feats = extractor.extract_features(lc_train, obj_id, meta)
        feats['target'] = meta['target']
        feats['split'] = split_name
        train_feats.append(feats)
    
    test_feats = []
    for obj_id in tqdm(test_meta['object_id'], desc=f'{split_name} test', leave=False):
        meta = test_meta[test_meta['object_id'] == obj_id].iloc[0].to_dict()
        feats = extractor.extract_features(lc_test, obj_id, meta)
        feats['split'] = split_name
        test_feats.append(feats)
    
    del lc_train, lc_test
    gc.collect()
    
    return pd.DataFrame(train_feats), pd.DataFrame(test_feats)

# Extract features
train_dfs, test_dfs = [], []
start = time.time()

for i, split in enumerate(all_splits, 1):
    print(f"[{i}/20] {split}...")
    tr, te = extract_split_features(split)
    train_dfs.append(tr)
    test_dfs.append(te)
    
    if i % 5 == 0:
        print(f"  Time: {(time.time()-start)/60:.1f}m")

train_features = pd.concat(train_dfs, ignore_index=True)
test_features = pd.concat(test_dfs, ignore_index=True)

train_features.to_csv('train_features_v3.csv', index=False)
test_features.to_csv('test_features_v3.csv', index=False)

print(f"\n✓ Features extracted! Train: {train_features.shape}, Test: {test_features.shape}")

[1/20] split_01...


split_01 train:   0%|          | 0/155 [00:00<?, ?it/s]

split_01 test:   0%|          | 0/364 [00:00<?, ?it/s]

[2/20] split_02...


split_02 train:   0%|          | 0/170 [00:00<?, ?it/s]

...

split_19 test:   0%|          | 0/375 [00:00<?, ?it/s]

[20/20] split_20...


split_20 train:   0%|          | 0/153 [00:00<?, ?it/s]

split_20 test:   0%|          | 0/358 [00:00<?, ?it/s]

  Time: 5.8m

✓ Features extracted! Train: (3043, 127), Test: (7135, 126)


In [4]:

# CELL 4: FEATURE ENGINEERING + SELECTION
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import mutual_info_classif
from sklearn.ensemble import RandomForestClassifier

def create_interaction_features(df, feature_cols):
    """Create domain-specific interactions"""
    df_new = df[feature_cols].copy()
    
    # TDE smoothness score
    if 'detection_rate' in df_new.columns and 'flux_mad' in df_new.columns:
        df_new['smoothness_score'] = df_new['detection_rate'] / (df_new['flux_mad'] + 1e-5)
    
    # Symmetry features
    if 'r_time_to_peak' in df_new.columns and 'r_time_from_peak' in df_new.columns:
        df_new['r_symmetry_score'] = 1 - np.abs(df_new['r_time_to_peak'] - df_new['r_time_from_peak']) / (df_new['duration'] + 1e-5)
    
    # Parabolic quality
    if 'r_rise_parabolic_fit' in df_new.columns and 'r_decline_parabolic_fit' in df_new.columns:
        df_new['parabolic_avg'] = (df_new['r_rise_parabolic_fit'] + df_new['r_decline_parabolic_fit']) / 2
    
    # Power-law TDE signature
    if 'r_rise_powerlaw' in df_new.columns and 'r_decline_powerlaw' in df_new.columns:
        # TDE signature: rise ~1.67, decline ~-1.67
        df_new['tde_powerlaw_score'] = (
            np.abs(df_new['r_rise_powerlaw'] - 1.67) + 
            np.abs(df_new['r_decline_powerlaw'] + 1.67)
        )
        df_new['tde_powerlaw_score'] = 1 / (1 + df_new['tde_powerlaw_score'])  # Lower deviation = higher score
    
    # Peak coherence (multi-band sync)
    if 'peak_time_std' in df_new.columns and 'duration' in df_new.columns:
        df_new['peak_coherence'] = 1 / (1 + df_new['peak_time_std'] / (df_new['duration'] + 1e-5))
    
    # Variability contrast
    for f1, f2 in [('r', 'i'), ('g', 'r')]:
        var1, var2 = f'{f1}_excess_var', f'{f2}_excess_var'
        if var1 in df_new.columns and var2 in df_new.columns:
            df_new[f'var_contrast_{f1}_{f2}'] = np.abs(df_new[var1] - df_new[var2])
    
    # SNR quality
    if 'snr_median' in df_new.columns and 'n_detections' in df_new.columns:
        df_new['quality_score'] = df_new['snr_median'] * df_new['detection_rate']
    
    return df_new

# Prepare data
meta_cols = ['object_id', 'split', 'target']
feature_cols = [c for c in train_features.columns if c not in meta_cols]

X_train = create_interaction_features(train_features, feature_cols)
X_test = create_interaction_features(test_features, feature_cols)
y_train = train_features['target'].values

feature_cols_v2 = X_train.columns.tolist()
print(f"Total features before selection: {len(feature_cols_v2)}")

# Impute + handle inf
imputer = SimpleImputer(strategy='median')
X_train_imp = imputer.fit_transform(X_train)
X_test_imp = imputer.transform(X_test)

X_train_imp = np.nan_to_num(X_train_imp, nan=0, posinf=0, neginf=0)
X_test_imp = np.nan_to_num(X_test_imp, nan=0, posinf=0, neginf=0)

# === FEATURE SELECTION ===
print("\n🔍 Feature Selection...")

# 1. Mutual Information
mi_scores = mutual_info_classif(X_train_imp, y_train, random_state=42, n_neighbors=5)
mi_df = pd.DataFrame({
    'feature': feature_cols_v2,
    'mi_score': mi_scores
}).sort_values('mi_score', ascending=False)

# 2. Random Forest Importance
rf = RandomForestClassifier(n_estimators=100, max_depth=10, 
                            class_weight='balanced', random_state=42, n_jobs=-1)
rf.fit(X_train_imp, y_train)
rf_importance = rf.feature_importances_

importance_df = pd.DataFrame({
    'feature': feature_cols_v2,
    'rf_importance': rf_importance
}).sort_values('rf_importance', ascending=False)

# 3. Combine scores
mi_norm = (mi_scores - mi_scores.min()) / (mi_scores.max() - mi_scores.min() + 1e-10)
rf_norm = (rf_importance - rf_importance.min()) / (rf_importance.max() - rf_importance.min() + 1e-10)
combined_score = mi_norm * 0.6 + rf_norm * 0.4

ranking = pd.DataFrame({
    'feature': feature_cols_v2,
    'mi_score': mi_scores,
    'rf_importance': rf_importance,
    'combined_score': combined_score
}).sort_values('combined_score', ascending=False)

# Select top 120 features 
TOP_K = 120
selected_features = ranking.head(TOP_K)['feature'].tolist()
selected_mask = np.isin(feature_cols_v2, selected_features)

X_train_imp = X_train_imp[:, selected_mask]
X_test_imp = X_test_imp[:, selected_mask]
feature_cols_v2 = selected_features

print(f" Selected TOP-{TOP_K} features")
print(f"\nTop 20 most important:")
for i, row in ranking.head(20).iterrows():
    print(f"  {row['feature'][:40]:40s} - Combined: {row['combined_score']:.4f}")

print(f"\n Final shape - Train: {X_train_imp.shape}, Test: {X_test_imp.shape}")

Total features before selection: 132

🔍 Feature Selection...
✓ Selected TOP-120 features

Top 20 most important:
  snr_median                               - Combined: 0.9305
  r_excess_var                             - Combined: 0.7999
  r_decline_powerlaw                       - Combined: 0.7314
  i_decline_powerlaw                       - Combined: 0.6977
  quality_score                            - Combined: 0.5593
  flux_median                              - Combined: 0.5381
  var_contrast_r_i                         - Combined: 0.5331
  flux_mad                                 - Combined: 0.5287
  g_excess_var                             - Combined: 0.5252
  g_flux_min                               - Combined: 0.4955
  r_decline_parabolic_fit                  - Combined: 0.4887
  beyond_1std                              - Combined: 0.4664
  detection_rate                           - Combined: 0.4590
  flux_p05                                 - Combined: 0.4427
  flux_skewness    

In [5]:

# OPTIMIZED LIGHTGBM WITH OPTUNA

import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score, precision_score, recall_score
import optuna
from xgboost import XGBClassifier  

def objective(trial):
    """Optuna objective để tối ưu F1 score"""
    params = {
        'objective': 'binary',
        'metric': 'binary_logloss',  
        'boosting_type': 'gbdt',
        'num_leaves': trial.suggest_int('num_leaves', 20, 80),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1, log=True),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.5, 0.9),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.5, 0.9),
        'bagging_freq': 5,
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 50),
        'min_child_weight': trial.suggest_float('min_child_weight', 1e-4, 1e-1, log=True),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-2, 10, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-2, 10, log=True),
        'max_depth': trial.suggest_int('max_depth', 4, 12),
        'scale_pos_weight': 2895 / 148,  # Class imbalance ratio
        'verbose': -1,
        'random_state': 42,
    }
    
    skf = StratifiedKFold(5, shuffle=True, random_state=42)
    f1_scores = []
    
    for train_idx, val_idx in skf.split(X_train_imp, y_train):
        X_tr, y_tr = X_train_imp[train_idx], y_train[train_idx]
        X_val, y_val = X_train_imp[val_idx], y_train[val_idx]
        
        train_data = lgb.Dataset(X_tr, y_tr)
        val_data = lgb.Dataset(X_val, y_val, reference=train_data)
        
        model = lgb.train(params, train_data, num_boost_round=1000,
                         valid_sets=[val_data], callbacks=[lgb.early_stopping(50, verbose=False)])
        
        pred = model.predict(X_val, num_iteration=model.best_iteration)
        
        # Optimize threshold
        thresholds = np.arange(0.1, 0.9, 0.05)
        f1s = [f1_score(y_val, (pred >= t).astype(int)) for t in thresholds]
        f1_scores.append(max(f1s))
    
    return np.mean(f1_scores)

# Run Optuna (comment out if too slow, use manual params below)
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50, show_progress_bar=True)
best_params = study.best_params
print(f"✓ Using params: {best_params}")

[I 2025-12-24 04:41:16,806] A new study created in memory with name: no-name-547a38bf-d55d-4f48-b0e1-93fbc4614b7c


  0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-12-24 04:41:20,377] Trial 0 finished with value: 0.48734779311933263 and parameters: {'num_leaves': 80, 'learning_rate': 0.09680795464801335, 'feature_fraction': 0.7384542668661269, 'bagging_fraction': 0.8326663715690257, 'min_child_samples': 17, 'min_child_weight': 0.0004798009163532098, 'reg_alpha': 0.010199331489925724, 'reg_lambda': 1.5988680284461414, 'max_depth': 7}. Best is trial 0 with value: 0.48734779311933263.
[I 2025-12-24 04:41:21,159] Trial 1 finished with value: 0.3023867855563772 and parameters: {'num_leaves': 41, 'learning_rate': 0.03860035430356852, 'feature_fraction': 0.6140342242842481, 'bagging_fraction': 0.774622170778092, 'min_child_samples': 35, 'min_child_weight': 0.02582253678980184, 'reg_alpha': 1.4446225182692534, 'reg_lambda': 1.8940750124791945, 'max_depth': 4}. Best is trial 0 with value: 0.48734779311933263.
[I 2025-12-24 04:41:35,028] Trial 2 finished with value: 0.5132650137808576 and parameters: {'num_leaves': 72, 'learning_rate': 0.0108442260

In [8]:

#STACKING ENSEMBLE (LightGBM + XGBoost)

from sklearn.metrics import classification_report, f1_score, precision_score, recall_score
from xgboost import XGBClassifier

N_FOLDS = 5
skf = StratifiedKFold(N_FOLDS, shuffle=True, random_state=42)

# LightGBM params - Optimized for F1
lgb_params = {
    **best_params,
    'objective': 'binary',
    'metric': 'binary_logloss',  
    'boosting_type': 'gbdt',
    'bagging_freq': 5,
    'scale_pos_weight': 2895 / 148,  
    'verbose': -1,
    'random_state': 42,
}

# XGBoost params - Optimized for F1
xgb_params = {
    'n_estimators': 2000,
    'learning_rate': 0.03,
    'max_depth': 8,
    'subsample': 0.7,
    'colsample_bytree': 0.7,
    'scale_pos_weight': 2895 / 148,
    'eval_metric': 'logloss',  
    'tree_method': 'hist',
    'enable_categorical': False
}

# Storage
oof_lgb = np.zeros(len(X_train_imp))
oof_xgb = np.zeros(len(X_train_imp))
test_lgb = np.zeros(len(X_test_imp))
test_xgb = np.zeros(len(X_test_imp))

lgb_models = []
xgb_models = []

print("=" * 70)
print("TRAINING STACKING ENSEMBLE: LightGBM + XGBoost")
print("=" * 70)

for fold, (train_idx, val_idx) in enumerate(skf.split(X_train_imp, y_train), 1):
    print(f"\n{'='*50}\nFOLD {fold}/{N_FOLDS}\n{'='*50}")
    
    X_tr, y_tr = X_train_imp[train_idx], y_train[train_idx]
    X_val, y_val = X_train_imp[val_idx], y_train[val_idx]
    
    # === LightGBM ===
    print("\n[LightGBM]")
    train_data = lgb.Dataset(X_tr, y_tr)
    val_data = lgb.Dataset(X_val, y_val, reference=train_data)
    
    lgb_model = lgb.train(
        lgb_params,
        train_data,
        num_boost_round=2000,
        valid_sets=[val_data],
        callbacks=[lgb.early_stopping(50, verbose=False), lgb.log_evaluation(500)]  # Early stop at 50 rounds
    )
    
    lgb_models.append(lgb_model)
    oof_lgb[val_idx] = lgb_model.predict(X_val, num_iteration=lgb_model.best_iteration)
    test_lgb += lgb_model.predict(X_test_imp, num_iteration=lgb_model.best_iteration) / N_FOLDS
    
    # === XGBoost ===
    print("\n[XGBoost]")
    xgb_model = XGBClassifier(**xgb_params)
    xgb_model.fit(
        X_tr, y_tr,
        eval_set=[(X_val, y_val)],
        early_stopping_rounds=50,  # Early stop at 50 rounds
        verbose=False
    )
    
    xgb_models.append(xgb_model)
    oof_xgb[val_idx] = xgb_model.predict_proba(X_val)[:, 1]
    test_xgb += xgb_model.predict_proba(X_test_imp)[:, 1] / N_FOLDS
    
    # Fold evaluation
    print(f"\nFold {fold} OOF Performance:")
    
    # Tìm threshold tối ưu cho F1 Score (QUAN TRỌNG!)
    ensemble_val = (oof_lgb[val_idx] * 0.6 + oof_xgb[val_idx] * 0.4)
    thresholds = np.arange(0.05, 0.95, 0.01)
    f1_scores = [f1_score(y_val, (ensemble_val >= t).astype(int)) for t in thresholds]
    best_thresh = thresholds[np.argmax(f1_scores)]  # Threshold maximize F1
    
    val_binary = (ensemble_val >= best_thresh).astype(int)
    f1 = f1_score(y_val, val_binary)
    prec = precision_score(y_val, val_binary, zero_division=0)
    rec = recall_score(y_val, val_binary, zero_division=0)
    
    print(f"  LGB F1: {f1_score(y_val, (oof_lgb[val_idx] >= best_thresh).astype(int)):.4f}")
    print(f"  XGB F1: {f1_score(y_val, (oof_xgb[val_idx] >= best_thresh).astype(int)):.4f}")
    print(f"  Ensemble F1: {f1:.4f} (Precision: {prec:.4f}, Recall: {rec:.4f})")

# ENSEMBLE 
print(f"\n{'='*70}")
print("FINAL ENSEMBLE RESULTS")
print(f"{'='*70}")

# Weighted ensemble: LightGBM 60% + XGBoost 40%
oof_ensemble = oof_lgb * 0.6 + oof_xgb * 0.4
test_ensemble = test_lgb * 0.6 + test_xgb * 0.4

# Tìm threshold tối ưu toàn cục cho F1 Score
thresholds = np.arange(0.05, 0.95, 0.01)
f1_scores = [f1_score(y_train, (oof_ensemble >= t).astype(int)) for t in thresholds]
best_threshold = thresholds[np.argmax(f1_scores)] 
best_f1 = max(f1_scores)

oof_binary = (oof_ensemble >= best_threshold).astype(int)

print(f"\nBest Threshold: {best_threshold:.3f}")
print(f"OOF F1 Score: {best_f1:.4f}")
print(f"\n{classification_report(y_train, oof_binary, target_names=['Non-TDE', 'TDE'])}")

# Confusion matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_train, oof_binary)
print(f"\nConfusion Matrix:")
print(f"              Predicted")
print(f"              Non-TDE  TDE")
print(f"Actual Non-TDE  {cm[0,0]:5d}  {cm[0,1]:5d}")
print(f"       TDE      {cm[1,0]:5d}  {cm[1,1]:5d}")

# Feature importance (from LightGBM)
importance = np.mean([m.feature_importance(importance_type='gain') for m in lgb_models], axis=0)
importance_df = pd.DataFrame({
    'feature': feature_cols_v2,
    'importance': importance
}).sort_values('importance', ascending=False)

print(f"\nTop 20 Features:")
print(importance_df.head(20).to_string(index=False))

TRAINING STACKING ENSEMBLE: LightGBM + XGBoost

FOLD 1/5

[LightGBM]
[500]	valid_0's binary_logloss: 0.127445

[XGBoost]

Fold 1 OOF Performance:
  LGB F1: 0.4528
  XGB F1: 0.4912
  Ensemble F1: 0.5185 (Precision: 0.5833, Recall: 0.4667)

FOLD 2/5

[LightGBM]

[XGBoost]

Fold 2 OOF Performance:
  LGB F1: 0.5417
  XGB F1: 0.5098
  Ensemble F1: 0.5306 (Precision: 0.6842, Recall: 0.4333)

FOLD 3/5

[LightGBM]
[500]	valid_0's binary_logloss: 0.120953

[XGBoost]

Fold 3 OOF Performance:
  LGB F1: 0.5185
  XGB F1: 0.5455
  Ensemble F1: 0.5818 (Precision: 0.6400, Recall: 0.5333)

FOLD 4/5

[LightGBM]

[XGBoost]

Fold 4 OOF Performance:
  LGB F1: 0.5915
  XGB F1: 0.5312
  Ensemble F1: 0.5797 (Precision: 0.5000, Recall: 0.6897)

FOLD 5/5

[LightGBM]

[XGBoost]

Fold 5 OOF Performance:
  LGB F1: 0.4839
  XGB F1: 0.3636
  Ensemble F1: 0.4828 (Precision: 0.4828, Recall: 0.4828)

FINAL ENSEMBLE RESULTS

Best Threshold: 0.300
OOF F1 Score: 0.5199

              precision    recall  f1-score   suppor

In [16]:

# FINAL SUBMISSION

import pandas as pd

# --- Use ensemble predictions directly ---
final_test_preds = test_ensemble 

# --- Prepare submission ---
test_ids = test_features['object_id'].values
submission_binary = (final_test_preds >= best_threshold).astype(int)

submission = pd.DataFrame({
    'object_id': test_ids,
    'prediction': submission_binary
})

submission.to_csv('submission_final.csv', index=False)

# --- Summary ---
print(f"\n{'='*70}")
print("SUBMISSION SUMMARY")
print(f"{'='*70}")
print(f"Model: LightGBM (60%) + XGBoost (40%) Ensemble")
print(f"Training samples: {len(X_train_imp):,}")
print(f"\nOOF Performance:")
print(f"  F1 Score: {best_f1:.4f}")
print(f"  Optimal Threshold: {best_threshold:.3f}")
print(f"\nSubmission:")
print(f"  Total objects: {len(submission):,}")
print(f"  Predicted TDEs: {submission_binary.sum()} ({submission_binary.mean()*100:.2f}%)")
print(f"  Expected TDEs: ~{len(submission) * 0.0486:.0f} (4.86%)")
print(f"\n✓ Submission saved as 'submission_final.csv'")
print(f"{'='*70}")

print("\nFirst 10 predictions:")
print(submission.head(10))
print("\nLast 10 predictions:")
print(submission.tail(10))


SUBMISSION SUMMARY
Model: LightGBM (60%) + XGBoost (40%) Ensemble
Training samples: 3,043

OOF Performance:
  F1 Score: 0.5199
  Optimal Threshold: 0.300

Submission:
  Total objects: 7,135
  Predicted TDEs: 415 (5.82%)
  Expected TDEs: ~347 (4.86%)

✓ Submission saved as 'submission_final.csv'

First 10 predictions:
                      object_id  prediction
0      Eluwaith_Mithrim_nothrim           0
1            Eru_heledir_archam           0
2             Gonhir_anann_fuin           0
3  Gwathuirim_haradrim_tegilbor           0
4              achas_minai_maen           0
5                 adab_fae_gath           0
6               adel_draug_gaur           0
7       aderthad_cuil_galadhrim           0
8           aegas_laug_ithildin           0
9              aegas_mereth_law           0

Last 10 predictions:
                  object_id  prediction
7125     tinuviel_hu_firiel           0
7126  tobas_lavan_glandagol           0
7127      tobas_maen_noroth           0
7128         u